In [9]:
import numpy as np
import plotly.graph_objects as go

from orbslam_results import ORBSLAMTrajectoryProcessor
from utils import create_scattermapbox, plot
from os import path

TRAJECTORY_NAME = 'vabadusepst'
# TRAJECTORY_NAME = 'oldtown'

processor = ORBSLAMTrajectoryProcessor(path.join('~/orbslam-out', TRAJECTORY_NAME))

trace_colors = {
   'gps': 'blue',
   'slam': 'red',
   'loc': ['dodgerblue', 'olive', 'orange']
}

fig_slam = go.Figure()

fig_slam.add_trace(
   go.Scatter(
      x=processor.orbslam.mapping.trajectory[:, 0], 
      y=processor.orbslam.mapping.trajectory[:, 2], 
      mode='markers', 
      name='SLAM (mapping)',
      marker=dict(color=trace_colors['slam'])
    )
)


for name, loc in processor.orbslam.localization.items():
    if (loc.trajectory.size > 0):
      fig_slam.add_trace(
         go.Scatter(
            x=loc.trajectory[:, 0], 
            y=loc.trajectory[:, 2], 
            mode='markers',
            marker=dict(color=trace_colors['loc'][name])
         )
      )

fig_slam.update_layout(yaxis=dict(scaleanchor="x", scaleratio=1), height=1200)
fig_slam.show()

In [11]:
processor.fit()

Trial 100 Complete [00h 00m 11s]
loss: 0.003045403864234686

Best loss So Far: 0.0011875237105414271
Total elapsed time: 00h 20m 32s
INFO:tensorflow:Oracle triggered exit
Epoch 1/400
191/191 [==============================] - 0s 686us/step - loss: 0.1504
Epoch 2/400
191/191 [==============================] - 0s 454us/step - loss: 0.0209
Epoch 3/400
191/191 [==============================] - 0s 452us/step - loss: 0.0080
Epoch 4/400
191/191 [==============================] - 0s 435us/step - loss: 0.0054
Epoch 5/400
191/191 [==============================] - 0s 451us/step - loss: 0.0042
Epoch 6/400
191/191 [==============================] - 0s 789us/step - loss: 0.0036
Epoch 7/400
191/191 [==============================] - 0s 424us/step - loss: 0.0033
Epoch 8/400
191/191 [==============================] - 0s 437us/step - loss: 0.0031
Epoch 9/400
191/191 [==============================] - 0s 415us/step - loss: 0.0030
Epoch 10/400
191/191 [==============================] - 0s 445us/step - l

In [12]:
from utils import umeyama_alignment


center_lat = np.mean(processor.orbslam.mapping.trajectory_gt_wgs[:, 0])
center_lon = np.mean(processor.orbslam.mapping.trajectory_gt_wgs[:, 1])

R, t, c = umeyama_alignment(processor.orbslam.mapping.trajectory.T,
                            processor.orbslam.mapping.trajectory_gt_utm.T,
                            True)
aligned_slam_trajectory_utm = np.array([t + c * R @ p for p in processor.orbslam.mapping.trajectory])
aligned_slam_trajectory_wgs = np.array([processor.orbslam.mapping.utm2wgs.transform(
    p[0], p[1], p[2])for p in aligned_slam_trajectory_utm])

traces = [
    create_scattermapbox(processor.orbslam.mapping.trajectory_gt_wgs, 'GPS (ground truth)', color=trace_colors['gps'], mode='lines'),
    create_scattermapbox(processor.orbslam.mapping.trajectory_wgs, 'fitted SLAM (mapping)', color=trace_colors['slam'], mode='lines'),
    create_scattermapbox(aligned_slam_trajectory_wgs, 'Umeyama SLAM (mapping)', color='green', mode='lines'),
]

for name, localization in processor.orbslam.localization.items():
    traces.append(create_scattermapbox(localization.trajectory_wgs, f'fitted loc_{name}', color=trace_colors['loc'][name]))

plot(traces,(center_lat, center_lon))
